# RAG

In [1]:
## setup
!pip install anthropic
!pip install voyageai
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install -U scikit-learn

In [23]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
os.environ['VOYAGE_API_KEY'] = ('VOYAGE_API_KEY')
openai.api_key = os.getenv("OPENAI_API_KEY")


# Initialize a Vector DB Class

In [24]:
import os
import pickle
import json
import numpy as np
import openai

class VectorDB:
    def __init__(self, name, api_key=None):
        self.name = name
        self.embeddings = []
        self.metadata = []
        self.query_cache = {}
        self.db_path = f"./data/{name}/vector_db.pkl"

    def load_data(self, data):
        if self.embeddings and self.metadata:
            print("Vector database is already loaded. Skipping data loading.")
            return
        if os.path.exists(self.db_path):
            print("Loading vector database from disk.")
            self.load_db()
            return

        texts = [f"Heading: {item['chunk_heading']}\n\n Chunk Text: {item['text']}" for item in data]
        self._embed_and_store(texts, data)
        self.save_db()
        print("Vector database loaded and saved.")

    def _embed_and_store(self, texts, data):
        batch_size = 128
        result = []

        for i in range(0, len(texts), batch_size):
            response = openai.Embedding.create(
                input=texts[i: i + batch_size],
                model="text-embedding-ada-002"  # Modelo recomendado para embeddings
            )
            embeddings = [res['embedding'] for res in response['data']]
            result.extend(embeddings)

        self.embeddings = result
        self.metadata = data

    def search(self, query, k=5, similarity_threshold=0.75):
        if query in self.query_cache:
            query_embedding = self.query_cache[query]
        else:
            response = openai.Embedding.create(
                input=[query],
                model="text-embedding-ada-002"
            )
            query_embedding = response['data'][0]['embedding']
            self.query_cache[query] = query_embedding

        if not self.embeddings:
            raise ValueError("No data loaded in the vector database.")

        # Cálculo da similaridade utilizando produto escalar
        similarities = np.dot(self.embeddings, query_embedding)
        top_indices = np.argsort(similarities)[::-1]
        top_examples = []

        for idx in top_indices:
            if similarities[idx] >= similarity_threshold:
                example = {
                    "metadata": self.metadata[idx],
                    "similarity": similarities[idx],
                }
                top_examples.append(example)

                if len(top_examples) >= k:
                    break
        self.save_db()
        return top_examples

    def save_db(self):
        data = {
            "embeddings": self.embeddings,
            "metadata": self.metadata,
            "query_cache": json.dumps(self.query_cache),
        }
        os.makedirs(os.path.dirname(self.db_path), exist_ok=True)
        with open(self.db_path, "wb") as file:
            pickle.dump(data, file)

    def load_db(self):
        if not os.path.exists(self.db_path):
            raise ValueError("Vector database file not found. Use load_data to create a new database.")
        with open(self.db_path, "rb") as file:
            data = pickle.load(file)
        self.embeddings = data["embeddings"]
        self.metadata = data["metadata"]
        self.query_cache = json.loads(data["query_cache"])


# Basic RAG

In [25]:
# Carregar o documento do seu caminho específico
with open('/home/lisamenezes/RAG-benchmark/data/fundamentos-train.json', 'r') as f:
    fundamentos_data = json.load(f)

# Inicializar o VectorDB com seus dados
db = VectorDB("fundamentos")
db.load_data(fundamentos_data)

def retrieve_base(query, db):
    results = db.search(query, k=3)
    context = ""
    for result in results:
        chunk = result['metadata']
        context += f"\n{chunk['text']}\n"
    return results, context

def answer_query_base(query, db):
    documents, context = retrieve_base(query, db)
    prompt = f"""
    You have been tasked with helping us to answer the following query: 
    <query>
    {query}
    </query>
    You have access to the following documents which are meant to provide context as you answer the query:
    <documents>
    {context}
    </documents>
    Please remain faithful to the underlying context, and only deviate from it if you are 100% sure that you know the answer already. 
    Answer the question now, and avoid providing preamble such as 'Here is the answer', etc.
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        max_tokens=2500,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    return response['choices'][0]['message']['content']



Loading vector database from disk.


In [26]:
# Exemplo de uso para realizar uma consulta e gerar uma resposta
query = "capacidade civil"
results, context = retrieve_base(query, db)

print("Contexto Recuperado:")
print(context)

# Gerar o texto usando o LLM
response = answer_query_base(query, db)

print("\nTexto Gerado pelo LLM:")
print(response)


Contexto Recuperado:

Art. 1oToda pessoa é capaz de direitos e deveres na ordem civil.

Art. 3oSão absolutamente 
		incapazes de exercer pessoalmente os atos da vida civil os menores de 16 
		(dezesseis) anos.(Redação 
		dada pela Lei nº 13.146, de 
2015)(Vigência)

Art. 4oSão incapazes, 
		relativamente a certos atos ou à maneira de os exercer:(Redação dada pela Lei nº 13.146, de 
2015)(Vigência) I - os maiores de dezesseis e menores de dezoito anos; II - os ébrios habituais e os viciados em 
		tóxico;(Redação dada pela Lei nº 13.146, de 
2015)(Vigência) III - aqueles que, por causa transitória ou 
		permanente, não puderem exprimir sua vontade;(Redação dada pela Lei nº 13.146, de 
2015)(Vigência) IV - os pródigos. Parágrafo único.  A capacidade dos indígenas 
		será regulada por legislação especial.(Redação dada pela Lei nº 13.146, de 
2015)(Vigência)


Texto Gerado pelo LLM:
A capacidade civil é a capacidade que toda pessoa possui para exercer direitos e deveres na ordem civil.


# Validação

In [ ]:
import time

def evaluate_model(test_data, db):
    results = []
    total_time = 0
    correct_answers = 0

    for item in test_data:
        query = item['chunk_heading']
        expected_text = item['text']

        # Medir tempo de inferência
        start_time = time.time()
        retrieved_docs, context = retrieve_base(query, db)
        end_time = time.time()

        # Comparar o texto recuperado com o esperado
        is_correct = expected_text.strip() in context.strip()

        # Atualizar métricas
        total_time += (end_time - start_time)
        correct_answers += int(is_correct)

        # Registrar resultados
        results.append({
            "query": query,
            "retrieved_context": context,
            "expected_text": expected_text,
            "is_correct": is_correct,
            "inference_time": end_time - start_time
        })

    # Calcular métricas gerais
    accuracy = correct_answers / len(test_data)
    average_time = total_time / len(test_data)

    return {
        "results": results,
        "accuracy": accuracy,
        "average_inference_time": average_time
    }

# Carregar os dados de teste
with open('/home/lisamenezes/RAG-benchmark/data/amostra.json', 'r') as f:
    test_data = json.load(f)

# Executar a avaliação
evaluation_results = evaluate_model(test_data, db)

# Exibir os resultados
print(f"Acurácia: {evaluation_results['accuracy'] * 100:.2f}%")
print(f"Tempo médio de inferência: {evaluation_results['average_inference_time']:.2f}s")


Acurácia: 100.00%
Tempo médio de inferência: 0.26s


In [30]:
evaluation_results

{'results': [{'query': 'capacidade civil',
   'retrieved_context': '\nArt. 1oToda pessoa é capaz de direitos e deveres na ordem civil.\n\nArt. 3oSão absolutamente \n\t\tincapazes de exercer pessoalmente os atos da vida civil os menores de 16 \n\t\t(dezesseis) anos.(Redação \n\t\tdada pela Lei nº 13.146, de \n2015)(Vigência)\n\nArt. 4oSão incapazes, \n\t\trelativamente a certos atos ou à maneira de os exercer:(Redação dada pela Lei nº 13.146, de \n2015)(Vigência) I - os maiores de dezesseis e menores de dezoito anos; II - os ébrios habituais e os viciados em \n\t\ttóxico;(Redação dada pela Lei nº 13.146, de \n2015)(Vigência) III - aqueles que, por causa transitória ou \n\t\tpermanente, não puderem exprimir sua vontade;(Redação dada pela Lei nº 13.146, de \n2015)(Vigência) IV - os pródigos. Parágrafo único. \xa0A capacidade dos indígenas \n\t\tserá regulada por legislação especial.(Redação dada pela Lei nº 13.146, de \n2015)(Vigência)\n',
   'expected_text': 'Art. 1oToda pessoa é capaz d

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_similarity(expected_text, retrieved_context):
    # Vetorização usando TF-IDF
    vectorizer = TfidfVectorizer().fit([expected_text, retrieved_context])
    tfidf_matrix = vectorizer.transform([expected_text, retrieved_context])
    
    # Similaridade de cosseno
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    return similarity

def evaluate_model_with_similarity(test_data, db, similarity_threshold=0.75):
    results = []
    total_time = 0
    correct_answers = 0

    for item in test_data:
        query = item['chunk_heading']
        expected_text = item['text']

        # Medir tempo de inferência
        start_time = time.time()
        retrieved_docs, context = retrieve_base(query, db)
        end_time = time.time()

        # Calcular a similaridade entre o texto esperado e o contexto recuperado
        similarity = calculate_similarity(expected_text, context)
        is_correct = similarity >= similarity_threshold

        # Atualizar métricas
        total_time += (end_time - start_time)
        correct_answers += int(is_correct)

        # Registrar resultados
        results.append({
            "query": query,
            "retrieved_context": context,
            "expected_text": expected_text,
            "similarity": similarity,
            "is_correct": is_correct,
            "inference_time": end_time - start_time
        })

    # Calcular métricas gerais
    accuracy = correct_answers / len(test_data)
    average_time = total_time / len(test_data)

    return {
        "results": results,
        "accuracy": accuracy,
        "average_inference_time": average_time
    }
# Carregar os dados de teste
with open('/home/lisamenezes/RAG-benchmark/data/amostra.json', 'r') as f:
    test_data = json.load(f)

# Executar a avaliação
evaluation_results = evaluate_model_with_similarity(test_data, db)

# Exibir os resultados
print(f"Acurácia: {evaluation_results['accuracy'] * 100:.2f}%")
print(f"Tempo médio de inferência: {evaluation_results['average_inference_time']:.2f}s")


Acurácia: 90.00%
Tempo médio de inferência: 0.26s
